# Handout #12 - Part A : The "Leave-One-Out" Concept, i.e. Jackknife

The “leave-one-out” concept in regression is implemented for two primary reasons.

1.   To understand the effect of a single observation on the regression model    
2.   To obtain a more accurate measure of the true predictive ability of the regression model.

It should be noted that the “leave-one-out” notion extends beyond regression problems.  This notion is more commonly known as **jackknife resampling**.  A snip-it of the wiki entry for jackknife resampling is provided here.  The natural extension of jackknife resampling would be “leave-several-out.”  This is known as cross-validation when the goal is understanding the predictive ability of a regression model.    

<p align="center"><img src="https://drive.google.com/uc?export=view&id=1dVTGpUBf8emEdMVhZlnrX4zGTeGxj_ih" width="75%" height="75%"></p>

Source: https://en.wikipedia.org/wiki/Jackknife_resampling

## Load Tidyverse

In [ ]:
#load tidyverse package
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.2.0     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()





---




## Simple Case: "Leave-One-Out" for a Mean

Consider the following simple data.frame that will be used for this handout.

$$\bf{LeaveOneOut} = \begin{bmatrix}
          2 & 1 \\
          3 & 2\\
          5 & 3\\
          8 & 4\\
          10 & 5\\
          \end{bmatrix}
$$

where, the first vector is labeled $\bf{y} = [2, 3, 5, 8, 10]$, and the second vector is labeled $\bf{x} = [1, 2, 3, 4, 5]$.

The following code will put this data into a data.frame named LeaveOneOut in R.

In [ ]:
LeaveOneOut <- data.frame(y=c(2,3,5,8,10), x = c(1,2,3,4,5))
LeaveOneOut

y  x
1  2 1
2  3 2
3  5 3
4  8 4
5 10 5

The summarize() function has been used repeated in this course to obtain basic summaries of a data.frame.  This function is part of tidyverse().

In [ ]:
(LeaveOneOut
  %>% summarize(Mean_of_y = mean(y))
)

Mean_of_y
1 5.6



---



The **mean()** function is part of base R and not part of tidyverse().  If we
try to pass an entire data.frame through the mean() function an "NA" is returned.  This happnes because R does *not* know what you want to compute the mean of.  For example, do you want of $\bf{y}$? do you want the mean of $\bf{x}$? do you want the mean of the first row?, do you want the mean of the second row?, do you want the mean of the first and third row?, etc.

In [ ]:
mean(LeaveOneOut)

Warning message in mean.default(LeaveOneOut):
“argument is not numeric or logical: returning NA”


[1] NA

There is an **apply()** helper function that can be used to assist with calculations across the columns (or rows) of a data.frame.  For example,

$$\mbox{apply(LeaveOneOut, 2, mean })$$

can be used to compute the mean across the *columns* of a data.frame, and


$$\mbox{apply(LeaveOneOut, 1, mean })$$
can be used to compute the mean across *rows* of a data.frame in R.



<p align="center"><img src="https://drive.google.com/uc?export=view&id=1tLoDpyaWteDyi5eOlYS4b-lQxy2guJhS" width="35%" height="35%"></p>

In [ ]:
#Using apply to get the mean for each column of the LeaveOneOut data.frame
apply(LeaveOneOut,2,mean)

y   x 
5.6 3.0

In [ ]:
#Using apply to get the mean for each row of the LeaveOneOut data.frame
apply(LeaveOneOut,1,mean)

[1] 1.5 2.5 4.0 6.0 7.5



---



Consider the following schematic which identifies the row or column indices of the $\bf{LeaveOneOut}$ data.frame.

<p align="center"><img src="https://drive.google.com/uc?export=view&id=1rgslUxxHzHOe-WTSuRL3jXdDhRHToDbS" width="50%" height="50%"></p>

Getting the mean of of a particular column of a data.frame, say $\bf{y}$, can be done in one of two ways with the mean() function.

1. Using the column name
$$\mbox{mean(LeaveOneOut\$y)}$$
2. Using the column index
$$\mbox{mean(LeaveOneOut[}\space\mbox{,1])}$$

In [ ]:
#Getting the mean using the column name
Mean_of_y <- mean(LeaveOneOut$y)
Mean_of_y

[1] 5.6

In [ ]:
#Getting the mean using the column index
Mean_of_y <- mean(LeaveOneOut[ ,1])
Mean_of_y

[1] 5.6

Getting the mean of a particular row of a data.frame can be done in one way with the mean() function.

1. ~~Using the row name~~
$$\space$$
2. Using the row index
$$\mbox{mean(LeaveOneOut[}\space\mbox{1,}\space\mbox{])}$$

In [ ]:
#Getting the mean using the row index
Mean_of_Row1 <- mean(LeaveOneOut[1,])
Mean_of_Row1

Warning message in mean.default(LeaveOneOut[1, ]):
“argument is not numeric or logical: returning NA”


[1] NA

**Oops, that did not work -- arg...**  The reason this did not work is becuases of something called data types.  Data types are important underlying element of a programming language.  The result from pulling off a column from a data.frame is a numeric vector which the mean() function requires.  However, the result from pulling off a row from a data.frame is another data.frame.  The str() function in R can be used to identify the structure of an object.

In [ ]:
#The structure of the resulting object when pulling off the column y
str(LeaveOneOut$y)
#The structure of the resulting object when pulling off the 1st column
str(LeaveOneOut[,1])

 num [1:5] 2 3 5 8 10
 num [1:5] 2 3 5 8 10


Next, let's look at the structure of the object when pulling off the 1st row.

In [ ]:
#The structure of the resulting object when pulling off the 1st row
str(LeaveOneOut[1,])

'data.frame':	1 obs. of  2 variables:
 $ y: num 2
 $ x: num 1


This is frustrating, but not all is lost as there exists a **as.numeric()** function which will force the object to be a numeric vector.

In [ ]:
str(as.numeric(LeaveOneOut[1,]))

 num [1:2] 2 1


Here, LeaveOneOut[1,] object is being forced into a numeric vector and the mean is computed on this numeric vector.

In [ ]:
#Getting the mean using the row index
Mean_of_Row1 <- mean(as.numeric(LeaveOneOut[1,]))
Mean_of_Row1

[1] 1.5

<table width='100%'><tr><td bgcolor="purple"><font color="white" size="+2">Task</font></td></tr></table>

## Task:  Questions

1.   Run the following code chunk, identify exactly what is being computed here?


In [ ]:
mean(LeaveOneOut[1:2, 1])

[1] 2.5

2.   Run the following code chunk, identify exactly what is being computed here?


In [ ]:
mean(LeaveOneOut$y[1:2])

[1] 2.5

3.   Run the following code chunk, identify exactly what is being computed here?


In [ ]:
mean(LeaveOneOut$y[c(1,3,4)])

[1] 5

4.   Run the following code chunk, identify exactly what is being computed here?


In [ ]:
mean(LeaveOneOut$y[-5])

[1] 4.5

<table width='100%'><tr><td bgcolor="purple"><font color="white" size="+2">&nbsp;</font></td></tr></table>

The minus argument can be used to *temporarily* withhold an observation from calculations done on an object.  For example, the following will calculate the mean of $\bf{y}$ without the $1^{st}$ observation included in the calculation.


In [ ]:
mean(LeaveOneOut[-1,1])

[1] 6.5

Extending this idea to all observations.

In [ ]:

               mean(LeaveOneOut[-1, 1])
               mean(LeaveOneOut[-2, 1])
               mean(LeaveOneOut[-3, 1])
               mean(LeaveOneOut[-4, 1])
               mean(LeaveOneOut[-5, 1])


[1] 6.5

[1] 6.25

[1] 5.75

[1] 5

[1] 4.5

Notice that "code statements are being repeated" which implies a **for()** loop may prove useful.  It should be noted that some do not like to write code with for() loops, and this issues is worse in R as R does not use a very efficient algorithm for its for() loops.  All that aside, a for() loop will be used here.

In [ ]:
for(i in 1:5){
               Mean_without_datavalue <- mean(LeaveOneOut[-i, 1])
               print(Mean_without_datavalue)
             }

[1] 6.5
[1] 6.25
[1] 5.75
[1] 5
[1] 4.5


In the above, the print() statment simply prints the outcomes to the screen.  A better approach would be to save the outcomes from each iteration into a object, say a data.frame.

In [ ]:
#Create an empty data.frame for the outcomes
Mean_Jackknife_DF <- data.frame(MeanWithoutRowi = numeric())

#Create the for() loop
for(i in 1:5){
               Mean_Jackknife_DF[i,1] <- mean(LeaveOneOut[-i, 2])
              # print(Mean_Jackknife_DF)
             }
#Print the out data.frame after the for() loop
Mean_Jackknife_DF

MeanWithoutRowi
1 3.50           
2 3.25           
3 3.00           
4 2.75           
5 2.50

## Writing a Custom Jackknife Function for the Mean

#A custom function with name Jackknife_Mean
Purpose: Obtain the jackknife estimate of the mean
Inputs: This function takes two inputs
  1. mydata:  the name of the data.frame
  2. which_column:  which column to use in the data.frame

# Outputs: This function returns one data.frame object
  1. A data.frame with the jackknife estimates of the mean without Row i

Consider the following code that will create a custom function in R.  A function is simply a collection of code.  A well-written function allows your code to be more flexible and have increased functionality.

#A function in R should:

*   Have a good name
*   Have clearly defined chunks of code with comments
*   Take input arguments/parameters
*   Return something



In [ ]:


Jackknife_Mean = function(mydata, which_column){

    Mean_Jackknife_DF <- data.frame(MeanWithoutRowi = numeric())
    for(i in 1:nrow(mydata)){
                   Mean_Jackknife_DF[i,1] <- mean(mydata[-i, which_column])
                 }
    return(Mean_Jackknife_DF)
}

After the Jackknife_Mean() function has been created, let's use our custom function to obtain the jackknife estimate of the mean.

In [ ]:
Jackknife_Mean(mydata=LeaveOneOut, which_column=2)

MeanWithoutRowi
1 3.50           
2 3.25           
3 3.00           
4 2.75           
5 2.50

<table width='100%'><tr><td bgcolor="purple"><font color="white" size="+2">Task</font></td></tr></table>

## Task:  Questions

5.   Modify the function call to Jackknife_Mean() to obtain the jackknife estimates on the 2nd column of LeaveOneOut instead of the first column.


6.   It is important to test a function before using it.  Consider the following testing situation with a modified data.frame called LeaveOneOut_Test.  Verify that the Jackknife_Mean function works for this test case.


In [ ]:
LeaveOneOut_Test <- data.frame(y=c(20,30,50,80, 100, 120), x = c(10,20,30,40, 50, 60))
LeaveOneOut_Test

y   x 
1  20 10
2  30 20
3  50 30
4  80 40
5 100 50
6 120 60

In [ ]:
Jackknife_Mean(mydata=LeaveOneOut_Test, which_column=2)

MeanWithoutRowi
1 40             
2 38             
3 36             
4 34             
5 32             
6 30

7.   Modify, in an appropriate way, the Jackknife_Mean() custom function so that it will correctly on the LeaveOneOut_Test data.frame.  


In [ ]:
#A custom function with name Jackknife_Mean
#Purpose: Obtain the jackknife estimate of the mean
#Inputs: This function takes two inputs
#  1. mydata:  the name of the data.frame
#  2. which_column:  which column to use in the data.frame
# Outputs: This function returns one data.frame object
#  1. A data.frame with the jackknife estimates of the mean without Row i

Jackknife_Mean = function(mydata, which_column){

    Mean_Jackknife_DF <- data.frame(MeanWithoutRowi = numeric())
    for(i in 1:5){
                   Mean_Jackknife_DF[i,1] <- mean(mydata[-i, which_column])
                 }
    return(Mean_Jackknife_DF)
}

8.   A contents of the Jackknife_Mean function can be seen by using the command print(Jackknife_Mean).  Notice that this function contains absolutely no comments -- comments are an important element to good code writing.  Return to the above code chuck and include comments for your updated Jackknife_Mean() function.  Reprint the contents of this function and confirm comments have now been included.

In [ ]:
print(Jackknife_Mean)

function(mydata, which_column){
    
    Mean_Jackknife_DF <- data.frame(MeanWithoutRowi = numeric())
    for(i in 1:5){
                   Mean_Jackknife_DF[i,1] <- mean(mydata[-i, which_column])
                 }
    return(Mean_Jackknife_DF)
}
<bytecode: 0x55db7f427cb0>


<table width='100%'><tr><td bgcolor="purple"><font color="white" size="+2">&nbsp;</font></td></tr></table>

## "Leave-One-Out" for Predicted Values from a Regression Model

One primary reason to consider the "leave-one-out" method for a regression model is to obtain a *better* measure of the true predictive ability of a regression model.  In particular, it is useful to determine how well your predictive model **cross-validates**.  

**Main Idea of Cross-Validating a Model**:  Withhold one (or more) observations from the model building process, then use the model to predict for the observation(s) that were withheld.


Reconsider the simple data.frame that was used above in this handout.

$$\bf{LeaveOneOut} = \begin{bmatrix}
          2 & 1 \\
          3 & 2\\
          5 & 3\\
          8 & 4\\
          10 & 5\\
          \end{bmatrix}
$$

where, the first vector is labeled $\bf{y} = [2, 3, 5, 8, 10]$, and the second vector is labeled $\bf{x} = [1, 2, 3, 4, 5]$.

For this simple situation, let's consider the regression model $y \sim x$.

In [ ]:
LinearModel_y_x <- lm(y~x, data = LeaveOneOut)
summary(LinearModel_y_x)


Call:
lm(formula = y ~ x, data = LeaveOneOut)

Residuals:
   1    2    3    4    5 
 0.6 -0.5 -0.6  0.3  0.2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)   
(Intercept)  -0.7000     0.6351  -1.102  0.35086   
x             2.1000     0.1915  10.967  0.00162 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6055 on 3 degrees of freedom
Multiple R-squared:  0.9757,	Adjusted R-squared:  0.9676 
F-statistic: 120.3 on 1 and 3 DF,  p-value: 0.001623


Make note of some of the model fitting statistics.

*  $RMSE$ value is $\hat{\sigma} = 0.6055$
*  $R^2$ value is $R^2 = 97.5\%$

The vector of predicted values from this model can be computed as $\bf{X \hat{\beta}}$

$$
\hat{E}{(Y|x)} = \bf{X \hat{\beta}} =
 \begin{bmatrix}
          1.4 \\
          3.5\\
          5.6\\
          7.7\\
          9.8\\
 \end{bmatrix}
$$



The vector of the residuals can be computed as follows

$$
\bf{Residual} = ( \bf{y} - \hat{E}{(Y|x)} )
$$



$$\bf{Residual} = \begin{bmatrix}
          2 \\
          3\\
          5\\
          8\\
          10\\
          \end{bmatrix} - \begin{bmatrix}
          1.4 \\
          3.5\\
          5.6\\
          7.7\\
          9.8\\
          \end{bmatrix} = \begin{bmatrix}
          +0.6 \\
          -0.5\\
          -0.6\\
          +0.3\\
          +0.2\\
          \end{bmatrix}
$$

In [ ]:
(LeaveOneOut
  %>% mutate(
             Predicted_y = predict(LinearModel_y_x, newdata=LeaveOneOut),
             Residual = (y - Predicted_y)
            )
) -> LeaveOneOut_with_PredictedandResiduals

head(LeaveOneOut_with_PredictedandResiduals)

y  x Predicted_y Residual
1  2 1 1.4          0.6    
2  3 2 3.5         -0.5    
3  5 3 5.6         -0.6    
4  8 4 7.7          0.3    
5 10 5 9.8          0.2



---



Consider the following code that will
1.  Fit the model $y \sim x$ without the $i^{th}$ observation
2.  Compute a predicted value for the $i^{th}$ observation with the model that excluded this observation from consideration
3.  Compute the corresponding residual for the $i^{the}$ observation

In [ ]:
#Create an empty data.frame for the outcomes
Jackknife_DF <- data.frame(y=numeric(), Predicted = numeric(), Residual=numeric())

#Create the for() loop
for(i in 1:5){
               LinearModel_y_x_withoutdatavalue <- lm(y~x, data = LeaveOneOut[-i,])
               Jackknife_DF[i,1] <- LeaveOneOut[i,1]
               Jackknife_DF[i,2] <- predict(LinearModel_y_x_withoutdatavalue, newdata=LeaveOneOut[i,])
               Jackknife_DF[i,3] <- (Jackknife_DF[i,1] - Jackknife_DF[i,2])
               print(Jackknife_DF)
             }
#Print the out data.frame after the for() loop
Jackknife_DF

  y Predicted Residual
1 2       0.5      1.5
  y Predicted   Residual
1 2  0.500000  1.5000000
2 3  3.714286 -0.7142857
  y Predicted   Residual
1 2  0.500000  1.5000000
2 3  3.714286 -0.7142857
3 5  5.750000 -0.7500000
  y Predicted   Residual
1 2  0.500000  1.5000000
2 3  3.714286 -0.7142857
3 5  5.750000 -0.7500000
4 8  7.571429  0.4285714
   y Predicted   Residual
1  2  0.500000  1.5000000
2  3  3.714286 -0.7142857
3  5  5.750000 -0.7500000
4  8  7.571429  0.4285714
5 10  9.500000  0.5000000


y  Predicted Residual  
1  2 0.500000   1.5000000
2  3 3.714286  -0.7142857
3  5 5.750000  -0.7500000
4  8 7.571429   0.4285714
5 10 9.500000   0.5000000

The three most common measures of model performance in a cross-valdiation setting are:

1. Root Mean Square Error = $RMSE = \sqrt{\frac{\sum (Residual)^2}{n}}$
2. Mean Absolute Error = $MAE = \frac{\sum |Residual|}{n}$
3. Mean Absolte Percent Error = $MAPE = 100 * \left( \frac{\sum |\frac{y-\hat{y}}{y}|}{n} \right)$

Computing each of these model performance metrics for the data considered here.

In [ ]:
(Jackknife_DF
  %>% summarize(
                  RMSE = sqrt(mean(Residual^2)),
                  MAE = mean(abs(Residual)),
                  MAPE = 100 * mean(abs( (y-Predicted)/y ))
               )
)

RMSE      MAE       MAPE    
1 0.8667615 0.7785714 24.83333

Notice that the jackknife RMSE (0.867) is *somewhat higher* than the RMSE from the model fit (0.61).  This is expected because the jackknife RMSE is using a *more true error term* as the observation being predicted is witheld while fitting the model for the jackknife approach.  Often MAE will be slighly lower than RMSE because RMSE using the squaring of the residuals.

The mean absolute percent error (MAPE) is somewhat high at 24.83%.

## TASK

1.  Complete a jackknife for a real data set that we've used in this class.

2. Write a custom function to implement a jackknife for a general simple linear regression model.



---



---
End of Document

